In [60]:
import pandas as pd
from os import path
import gensim
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence

import glob, os, re
import numpy as np
import pandas as pd

import tensorflow as tf

from sklearn.model_selection import train_test_split


In [61]:
lyrics_data = 'songs_rawdata\인디_5000곡_240402.csv'

text_filepath = lyrics_data.split('\\')[-1].split('_')[0]

print(text_filepath)

인디


In [64]:
%%time
lyrics_data = 'songs_rawdata\인디_5000곡_240402.csv'

text_filepath = lyrics_data.split('\\')[-1].split('.csv')[0] + '.txt'
bigram_filepath = lyrics_data.split('\\')[-1].split('.csv')[0] + '_bigram.txt'
bigram_model_filepath = lyrics_data.split('\\')[-1].split('_')[0] + '_bigram_model'
corpus_filepath = lyrics_data.split('\\')[-1].split('.csv')[0] + '_bigram_corpus.csv'

indi = pd.read_csv(lyrics_data)
with open(text_filepath, 'w', encoding = 'utf-8') as f:
    for lyrics in indi.Lyrics.values:
        if pd.isnull(lyrics):
            continue
        f.write(lyrics+'\n')

CPU times: total: 31.2 ms
Wall time: 145 ms


In [65]:
# bigram_model_filepath = 'bigram_model'
sentences_normalized_path = text_filepath
unigram_sentences = LineSentence(sentences_normalized_path)
bigram_model = Phrases(unigram_sentences)
bigram_model.save(bigram_model_filepath)

In [54]:
%%time
bigram_sentences_filepath = bigram_filepath
with open(bigram_sentences_filepath, 'w', encoding = 'utf-8') as f:
    for unigram_sentence in unigram_sentences:
        bigram_sentence = bigram_model[unigram_sentence]
        f.write(' '.join(bigram_sentence) + '\n')

CPU times: total: 1.03 s
Wall time: 2.09 s


In [55]:
txt_path = bigram_filepath
raw_corpus = []

with open(txt_path, 'r', encoding = 'utf-8') as f:
    raw = f.read().splitlines()
    raw_corpus.extend(raw)
print(len(raw_corpus))

210235


In [56]:
def preprocess_sentence(sentence):
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z가-힣?.!,¿,_]+", " ", sentence)
    sentence = sentence.strip()
    sentence = re.sub(r"\(.\)", " ", sentence) # 7
    sentence = '<start> ' + sentence + ' <end>'
    return sentence

In [57]:
corpus = []
for sentence in raw_corpus:
    if len(sentence) == 0:
        continue
    if sentence[-1] == ':':
        continue
    if len(sentence)>150:
        continue
    
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
    
print(len(corpus))
print(corpus[:10])

210226
['<start> 돌아서는 너를 보며 <end>', '<start> 난 아무_말도 할_수 없었고 <end>', '<start> 슬퍼하기엔 짧았던 <end>', '<start> 나의 해는 저물어 갔네 <end>', '<start> 지나치는 모진 기억이 <end>', '<start> 바람_따라 흩어질 때면 <end>', '<start> 아무_일도 없듯이 보내주려 해 <end>', '<start> 아픈 맘이 남지 않도록 <end>', '<start> 안녕 멀어지는 나의 하루야 <end>', '<start> 빛나지 못한 나의 별들아 <end>']


In [58]:
df = pd.DataFrame({'col':corpus})
df.to_csv(corpus_filepath)